In [3]:
!pip install pandas pillow python-docx qrcode[pil]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import qrcode
from PIL import Image, ImageDraw, ImageFont
from docx import Document
from docx.shared import Inches
from google.colab import files

In [6]:
# carrega a planilha enviada
uploaded = files.upload()

# lê a planilha
df = pd.read_excel(next(iter(uploaded)))

Saving planilha.xlsx to planilha (1).xlsx


In [7]:
# cria o documento word
doc = Document()

# cria uma tabela com 3 colunas no documento Word
table = doc.add_table(rows=1, cols=3)

In [8]:
# upload da fonte .ttf a ser utilizada
uploaded = files.upload()

# caminho para a fonte ttf
font_path = 'Roboto-Regular.ttf'

# tamanho da fonte
font_size = 20

Saving Roboto-Regular.ttf to Roboto-Regular.ttf


In [9]:
from docx.enum.text import WD_ALIGN_PARAGRAPH

# cria uma tabela com 3 colunas no documento Word
table = doc.add_table(rows=1, cols=3)

# gera os QR Codes e adiciona ao documento Word
col_index = 0
for index, row in df.iterrows():
    nome = row['Nome']               # coloca o nome da coluna
    linkedin_link = row['Linkedin']

    # gera o QR Code
    qr = qrcode.make(linkedin_link)

    # adiciona o nome embaixo do QR Code
    qr_img = qr.convert("RGB")
    draw = ImageDraw.Draw(qr_img)
    font = ImageFont.truetype(font_path, font_size)

    # cria uma nova imagem maior para incluir o nome abaixo do QR Code
    qr_width, qr_height = qr_img.size
    new_img = Image.new("RGB", (qr_width, qr_height + 50), "white")
    new_img.paste(qr_img, (0, 0))

    # adiciona o nome abaixo do QR Code usando textbbox
    draw = ImageDraw.Draw(new_img)
    text_bbox = draw.textbbox((0, 0), nome, font=font)
    text_width = text_bbox[2] - text_bbox[0]  # Largura do texto
    draw.text(((qr_width - text_width) // 2, qr_height + 10), nome, font=font, fill="black")

    # salva a imagem temporariamente
    temp_image_path = f"qr_temp_{index}.png"
    new_img.save(temp_image_path)

    # adiciona a imagem à célula da tabela sem o parágrafo adicional
    cell = table.cell(0, col_index)
    run = cell.paragraphs[0].add_run()
    run.add_picture(temp_image_path, width=Inches(2))

    # incrementa o índice da coluna
    col_index += 1

    # se já adicionou 3 QR Codes (3 colunas), iniciar uma nova linha
    if col_index == 3:
        col_index = 0
        table.add_row()


In [10]:
# salva o documento Word
doc.save('QR_Codes_LinkedIn.docx')

In [11]:
# salva o arquivo gerado em word
files.download('QR_Codes_LinkedIn.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>